In [2]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
import altair as alt

In [3]:
def get_fiis_url(full=True,my_fiis=[]):
    r = requests.get('https://fiis.com.br/lista-de-fundos-imobiliarios/')
    soup = BeautifulSoup(r.text, 'html.parser')
    fii_url_list = []
    fii_html_list = soup.find(id='items-wrapper').find_all('a')
    
    if full:
        for fii_url in fii_html_list:
            fii_url_list.append(fii_url.get('href'))
 
        return fii_url_list
    else:
        for fii in fii_html_list:
            fii_name = fii.contents[1].contents[0]
            if fii_name in my_fiis:
                fii_url_list.append(fii.get('href'))
        
        return fii_url_list

In [4]:
def get_fii_news(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    fii_news = {}
    fii_news['news'] = []
    
    news_block = soup.find(id='news--wrapper').find_all('li')
    for news in news_block:
        news_dict = {}
        
        news_date = news.find(class_='date').contents[0]
        if len(news.find(class_='title')) == 1:
            news_title = news.find(class_='title').contents[0]
        else:
            news_title = ''
            for piece in news.find(class_='title'):
                if str(piece) != '<br/>':
                    news_title = news_title+f'{piece} '
                    
            news_title = news_title.rstrip()
            
        if news.find('a').get('href') != 'javascript:;':
            news_link_url = news.find('a').get('href')
        else:
            news_link_url = 'no_link'
        
        news_dict['date'] = news_date
        news_dict['title'] = news_title
        news_dict['link'] = news_link_url
        fii_news['news'].append(news_dict)
    
    
    return fii_news

In [5]:
def get_fii_info(url,news=False):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    fii_info = {}
    
    fii_kpis = soup.find(id='informations--indexes').find_all(class_='item')
    fii_admin_top = soup.find(id='informations--admin').find(class_='top-content')
    fii_admin_bottom = soup.find(id='informations--admin').find(class_='bottom-content').find_all(class_='item')   
    fii_basic = soup.find(id='informations--basic').find_all(class_='wrapper')
    fii_basic_1_items = fii_basic[0].find_all(class_='item')
    fii_basic_2_items = fii_basic[1].find_all(class_='item')
    
   
    fii_info["fiiCode"] = soup.find(id='fund-ticker').contents[0]
    fii_info["fiiFullName"] = soup.find(id='fund-name').contents[0].rstrip().lstrip()
    fii_info["currentValue"] = float(soup.find(class_='item quotation').find(class_='value').contents[0].replace(',','.'))
    
    fii_info["admin"] = {}
    fii_info["admin"]["name"] = fii_admin_top.find(class_='administrator-name').contents[0]
    fii_info["admin"]["cnpj"] = fii_admin_top.find(class_='administrator-doc').contents[0]
    fii_info["admin"]["phone"] = fii_admin_bottom[0].find(class_='value').contents[0]
    fii_info["admin"]["email"] = fii_admin_bottom[1].find(class_='value').find('a').contents[0].contents[0]
    fii_info["admin"]["site"] = fii_admin_bottom[2].find(class_='value').find('a').contents[0]
    
    
    fii_info["fiiBasicInfo"] = {}
    fii_info["fiiBasicInfo"]["stockName"] = fii_basic_1_items[0].find(class_='value').contents[0]
    fii_info["fiiBasicInfo"]["fiiType"] = fii_basic_1_items[1].find(class_='value').contents[0]
    fii_info["fiiBasicInfo"]["typeANBIMA"] = fii_basic_1_items[2].find(class_='value').contents[0]
    fii_info["fiiBasicInfo"]["CVM"] = fii_basic_1_items[3].find(class_='value').contents[0]
    fii_info["fiiBasicInfo"]["numberOfQuotas"] = int(fii_basic_2_items[0].find(class_='value').contents[0].replace('.',''))
    fii_info["fiiBasicInfo"]["numberOfQuotaHolders"] = int(fii_basic_2_items[1].find(class_='value').contents[0].replace('.',''))
    fii_info["fiiBasicInfo"]["fiiCNPJ"] = fii_basic_2_items[2].find(class_='value').contents[0]
    
    
    fii_info["DY"] = float(fii_kpis[0].find(class_='value').contents[0].replace(',','.'))
    
    fii_info["lastPayment"] = {}
    fii_info["lastPayment"]["currency"] = fii_kpis[1].find(class_='value').find(class_='currency').contents[0]
    fii_info["lastPayment"]["value"] = float(fii_kpis[1].find(class_='value').contents[1].replace(',','.'))
    
    fii_info["netPatrimony"] = {}
    fii_info["netPatrimony"]["currency"] = fii_kpis[2].find(class_='value').find(class_='currency').contents[0]
    fii_info["netPatrimony"]["value"] = fii_kpis[2].find(class_='value').contents[1]
   
    fii_info["valuePerQuota"] = {}
    fii_info["valuePerQuota"]["currency"] = fii_kpis[3].find(class_='value').find(class_='currency').contents[0]
    fii_info["valuePerQuota"]["value"] = float(fii_kpis[3].find(class_='value').contents[1].replace(',','.'))
        
    if news:
        fii_info['news'] = get_fii_news(url)

    return fii_info

In [6]:
def fix_na_value(value):
    if value.get('data-order') == '-9999999999' or value.get('data-order') == '9999999999':
        return None
    else:
        return float(value.get('data-order'))

def get_fiis_attributes_table():
    url = 'https://www.fundsexplorer.com.br/ranking'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')


    fiis_table_dict = []

    table_rows = soup.find_all('tr')

    for table_row in table_rows:
        fii_row_dict = {}
        attributes = table_row.find_all('td')

        if len(attributes) > 0:
            fii_row_dict["code"] = attributes[0].find('a').contents[0]
            fii_row_dict["url"] = attributes[0].find('a').get('href')
            fii_row_dict["type"] = attributes[1].contents[0]
            fii_row_dict["current_price"] = fix_na_value(attributes[2])
            fii_row_dict["dividend"] = fix_na_value(attributes[3])

            fii_row_dict["dy_current"] = fix_na_value(attributes[5])
            fii_row_dict["dy_3m_sum"] = fix_na_value(attributes[6])
            fii_row_dict["dy_6m_sum"] = fix_na_value(attributes[7])
            fii_row_dict["dy_12m_sum"] = fix_na_value(attributes[8])
            fii_row_dict["dy_3m_avg"] = fix_na_value(attributes[9])
            fii_row_dict["dy_6m_avg"] = fix_na_value(attributes[10])
            fii_row_dict["dy_12m_avg"] = fix_na_value(attributes[11])
            fii_row_dict["dy_annual"] = fix_na_value(attributes[12])

            fii_row_dict["price_variation"] = fix_na_value(attributes[13])

            fii_row_dict["profitability_period"] = fix_na_value(attributes[14])
            fii_row_dict["profitability_sum_ytd"] = fix_na_value(attributes[15])


            fii_row_dict["patrimony_net_value"] = fix_na_value(attributes[16])
            fii_row_dict["patrimony_vpa"] = fix_na_value(attributes[17])
            fii_row_dict["patrimony_p_vpa"] = fix_na_value(attributes[18])
            fii_row_dict["patrimony_dy"] = fix_na_value(attributes[19])
            fii_row_dict["patrimony_variation"] = fix_na_value(attributes[20])

            fii_row_dict["patrimony_profitability_period"] = fix_na_value(attributes[21])
            fii_row_dict["patrimony_profitability_sum_ytd"] = fix_na_value(attributes[22])

            fii_row_dict["vacancy_physical"] = fix_na_value(attributes[23])
            fii_row_dict["vacancy_financial"] = fix_na_value(attributes[24])


            fii_row_dict["realty"] = int(attributes[25].get('data-order'))


            fiis_table_dict.append(fii_row_dict)
    return fiis_table_dict

In [7]:
def original_red_box(dataframe):
    plot = alt.Chart().mark_circle(size=60).encode(
        x='dy_12m_sum',
        y='patrimony_p_vpa',
        color='type',
        tooltip=['code', 'type', 'current_price', 'dy_current', 'patrimony_p_vpa', 'dy_12m_sum'],
        size='current_price'
    ).interactive()


    x_line = alt.Chart().mark_rule(color='red').encode(
        x='red_x:Q',
        size=alt.value(1)
    )

    y_line = alt.Chart().mark_rule(color='red').encode(
        y='red_y:Q',
        size=alt.value(1)
    )

    return alt.layer(
        plot, x_line, y_line,
        data=dataframe
    ).transform_calculate(
        red_x="8",
        red_y="1.2"
    )


In [8]:
# with open("../data/fiis_table.json", "w") as outfile:  
#     json.dump(get_fiis_attributes_table(), outfile)

In [9]:
my_fiis_json = "../data/my_fiis.json"


with open(my_fiis_json) as f:
  my_fiis_dict = json.load(f)

In [10]:
fiis_df = pd.DataFrame.from_dict(get_fiis_attributes_table())
my_fiis_df = fiis_df[fiis_df["code"].isin(my_fiis_dict["fiis"])]
fiis_df.describe

<bound method NDFrame.describe of        code            url                 type  current_price  dividend  \
0    FIVN11  /funds/fivn11            Shoppings           4.56     331.0   
1    BZLI11  /funds/bzli11  Títulos e Val. Mob.          14.80   80926.0   
2    XTED11  /funds/xted11   Lajes Corporativas           7.28    3427.0   
3    ALMI11  /funds/almi11   Lajes Corporativas        1195.15     105.0   
4    DOMC11  /funds/domc11   Lajes Corporativas         475.89       5.0   
..      ...            ...                  ...            ...       ...   
178  FCFL11  /funds/fcfl11               Outros         108.78    1930.0   
179  XPML11  /funds/xpml11            Shoppings         109.45  101425.0   
180  ALZR11  /funds/alzr11            Logística         128.26    9091.0   
181  FLRP11  /funds/flrp11            Shoppings        1394.00      14.0   
182  CNES11  /funds/cnes11   Lajes Corporativas          64.15     263.0   

     dy_current  dy_3m_sum  dy_6m_sum  dy_12m_sum  dy

In [11]:
red_box_df = fiis_df[fiis_df["patrimony_p_vpa"]<=1.2]
red_box_df = red_box_df[red_box_df["dy_12m_sum"]>=8].sort_values(ascending=True,by=["code"])
red_box_df.head(100)

,code,url,type,current_price,dividend,dy_current,dy_3m_sum,dy_6m_sum,dy_12m_sum,dy_3m_avg,...,patrimony_net_value,patrimony_vpa,patrimony_p_vpa,patrimony_dy,patrimony_variation,patrimony_profitability_period,patrimony_profitability_sum_ytd,vacancy_physical,vacancy_financial,realty
89,BARI11,/funds/bari11,Títulos e Val. Mob.,103.50,62057.0,0.713855,2.034328,4.168580,8.758549,0.678109,...,2.214571e+08,99.121845,1.044169,0.716290,-0.132692,0.582647,3.946564,NaN,NaN,0
52,BBFI11B,/funds/bbfi11b,Lajes Corporativas,2299.90,184.0,1.102383,2.926600,6.083454,11.492221,0.975533,...,4.080523e+08,3138.864010,0.732717,0.806892,0.033097,0.840256,5.457429,72.2,NaN,2
95,BCRI11,/funds/bcri11,Títulos e Val. Mob.,108.15,5455.0,0.694698,2.104932,4.408450,8.088260,0.701644,...,3.147348e+08,105.622841,1.023926,0.719541,-0.428728,0.287728,5.165827,NaN,NaN,0
70,CTXT11,/funds/ctxt11,Lajes Corporativas,48.05,242.0,0.783505,2.290332,4.527468,8.312590,0.763444,...,1.384875e+08,53.381881,0.900118,0.711852,0.053209,0.765440,4.597569,8.6,NaN,1
55,CXCE11B,/funds/cxce11b,Outros,99.97,329.0,0.815658,2.404465,4.976711,9.762065,0.801488,...,1.477014e+08,86.226339,1.159391,0.944058,-96.003368,-95.965638,-95.764890,0.0,NaN,1
139,DMAC11,/funds/dmac11,Residencial,373.80,124.0,0.709059,2.239657,4.526488,8.917548,0.746552,...,9.770550e+07,625.884033,0.597235,0.396368,1.549552,1.952062,-13.004218,NaN,NaN,0
46,FAMB11B,/funds/famb11b,Lajes Corporativas,3103.10,54.0,0.810071,2.608680,5.809861,11.117287,0.869560,...,3.705861e+08,3536.126965,0.877542,0.733069,0.535149,1.272141,9.163311,0.0,NaN,1
147,FEXC11,/funds/fexc11,Títulos e Val. Mob.,102.00,2787.0,0.582298,2.379832,4.892762,8.308096,0.793277,...,2.413455e+08,98.768992,1.032713,0.607478,-0.116830,0.489938,3.391963,NaN,NaN,0
30,FIIP11B,/funds/fiip11b,Logística,185.00,349.0,0.286817,6.373338,7.909158,11.943116,2.124446,...,1.835493e+08,197.968929,0.934490,0.286717,0.402443,0.690313,11.187823,0.0,NaN,7
152,HCTR11,/funds/hctr11,Outros,128.40,10948.0,0.918439,2.680838,6.322899,12.121855,0.893613,...,2.253778e+08,117.149554,1.096035,0.998723,0.503771,1.507525,10.318771,NaN,NaN,0


In [12]:
red_box_plot = original_red_box(red_box_df)
red_box_plot

alt.LayerChart(...)

In [13]:
my_fiis_df = my_fiis_df.sort_values(ascending=True,by=["code"])
my_fiis_df.head(100)

,code,url,type,current_price,dividend,dy_current,dy_3m_sum,dy_6m_sum,dy_12m_sum,dy_3m_avg,...,patrimony_net_value,patrimony_vpa,patrimony_p_vpa,patrimony_dy,patrimony_variation,patrimony_profitability_period,patrimony_profitability_sum_ytd,vacancy_physical,vacancy_financial,realty
145,BCFF11,/funds/bcff11,Títulos e Val. Mob.,88.70,47037.0,0.532544,1.407692,2.918359,6.244774,0.469231,...,1.721999e+09,83.742706,1.059197,0.537360,-1.562956,-1.033994,-19.106697,NaN,NaN,0
95,BCRI11,/funds/bcri11,Títulos e Val. Mob.,108.15,5455.0,0.694698,2.104932,4.408450,8.088260,0.701644,...,3.147348e+08,105.622841,1.023926,0.719541,-0.428728,0.287728,5.165827,NaN,NaN,0
121,BPFF11,/funds/bpff11,Títulos e Val. Mob.,85.05,5392.0,0.606061,1.688411,3.591117,7.962888,0.562804,...,2.496559e+08,93.544026,0.909198,0.577268,3.536230,4.133912,-10.032665,NaN,NaN,0
90,CBOP11,/funds/cbop11,Lajes Corporativas,74.45,1254.0,0.680650,1.695262,3.336538,6.793236,0.565087,...,1.215940e+08,85.932182,0.866381,0.570217,0.050741,0.621248,4.166529,10.37,10.87,1
130,CEOC11,/funds/ceoc11,Lajes Corporativas,83.00,1654.0,0.687775,1.952845,3.923734,7.606762,0.650948,...,1.780693e+08,98.072208,0.846315,0.582776,-0.049699,0.532787,3.694782,0.00,NaN,1
147,FEXC11,/funds/fexc11,Títulos e Val. Mob.,102.00,2787.0,0.582298,2.379832,4.892762,8.308096,0.793277,...,2.413455e+08,98.768992,1.032713,0.607478,-0.116830,0.489938,3.391963,NaN,NaN,0
59,HFOF11,/funds/hfof11,Títulos e Val. Mob.,113.50,23545.0,0.429185,2.190335,4.260027,8.411303,0.730112,...,1.668276e+09,98.973620,1.146770,0.505185,-3.430960,-2.943107,-4.551709,NaN,NaN,0
50,HGCR11,/funds/hgcr11,Títulos e Val. Mob.,103.49,22012.0,0.479138,1.460895,3.216844,6.508232,0.486965,...,1.308826e+09,105.785514,0.978300,0.453748,0.356137,0.811501,3.500952,NaN,NaN,0
157,IRDM11,/funds/irdm11,Títulos e Val. Mob.,115.97,41037.0,0.732849,2.182713,4.348447,9.025880,0.727571,...,1.030185e+09,97.105321,1.194270,0.871672,-1.481923,-0.623168,-1.623605,NaN,NaN,0
151,JRDM11,/funds/jrdm11,Shoppings,75.32,83.0,0.215822,0.215822,0.607616,3.897005,0.071941,...,2.281562e+08,90.431679,0.832894,0.178332,0.018122,0.196486,0.637825,2.21,NaN,1


In [14]:
my_fiis_plot = original_red_box(my_fiis_df)
my_fiis_plot

alt.LayerChart(...)

In [15]:
my_fiis_df[['current_price','dy_current','dividend',
            'dy_3m_sum','dy_6m_sum','dy_12m_sum',
            'dy_3m_avg','dy_6m_avg','dy_12m_avg',
            'patrimony_p_vpa',
            'vacancy_physical','vacancy_financial']].describe()

,current_price,dy_current,dividend,dy_3m_sum,dy_6m_sum,dy_12m_sum,dy_3m_avg,dy_6m_avg,dy_12m_avg,patrimony_p_vpa,vacancy_physical,vacancy_financial
count,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,7.000000,1.00
mean,89.327273,0.632083,43353.227273,1.902985,3.962549,7.537581,0.634328,0.660425,0.628132,1.004431,3.082857,10.87
std,24.091657,0.270001,113200.606973,0.727540,1.479681,2.896780,0.242513,0.246613,0.241398,0.090710,3.801599,NaN
min,10.570000,0.215822,83.000000,0.215822,0.607616,0.000000,0.071941,0.101269,0.000000,0.832894,0.000000,10.87
25%,80.697500,0.490458,5357.500000,1.592119,3.369027,6.701180,0.530706,0.561504,0.558432,0.956027,0.000000,10.87
50%,96.255000,0.600903,11839.000000,1.882923,3.902832,7.532707,0.627641,0.650472,0.627726,1.020978,2.210000,10.87
75%,103.382500,0.685994,38849.500000,2.099377,4.326342,8.385501,0.699792,0.721057,0.698792,1.054868,4.500000,10.87
max,115.970000,1.626425,543002.000000,4.318868,9.127177,16.694642,1.439623,1.521196,1.391220,1.194270,10.370000,10.87


# FIIs in crysis
Observation of indicators of crysis over FIIs

## Dividend = 0%

## High Vacancy
### Pyshical or Financial vacancy over 50%

## Price Variation (going down)
### Price closure about the past two months

# High performance FIIs
Observation of kpis poiting to high performaces FIIs

## RedBox Method (by GuiaInvest)
### V/PA vs DY last 12 months

## Price Variation (going up + nice DY)
### Price closure about the past two months

# KPIs per FII types
Analyses based on the FIIs types

## Top 3 average price variation (going up)

## Top 3 average DY

## Top 3 average vacancy